In [1]:
import praw 
import pandas as pd

In [2]:
reddit = praw.Reddit(
    client_id="cXb7-vX9BLku6yVl6aKagg",
    client_secret="Pa1gMfMKKUqS_hyFllKzzY6d7__JPw",
    user_agent="jellyfish",
)

In [4]:
posts=pd.read_csv('suicide.csv')
posts=posts.loc[(posts.key_words=='suicide') ]    #Only analyze 'suicide', narrow the range.
posts=posts.sort_values(by='score', ascending=False)  #Score means upvoting numbers
posts_sub=posts.loc[posts.subreddit=='SuicideWatch'].sort_values(by='score', ascending=False)
posts_sub
posts.head(50)

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created,key_words
5399,5400,to block the German autobahn,79603,zdc3sl,therewasanattempt,https://v.redd.it/zie3h567h54a1,4152,NaN,1.670259e+09,suicide
1614,1614,Gabby Petito Confirmed Dead by her Father on T...,75330,prift6,awfuleverything,https://i.redd.it/m3cc2ot6ijo71.png,11510,NaN,1.632092e+09,suicide
5884,5885,"Print("" suicidal thoughts"")",74839,e4golj,dankmemes,https://i.redd.it/4r4iepqfj0241.jpg,405,NaN,1.575202e+09,suicide
5376,5377,to make his death look like a suicide,61442,gd0bzc,therewasanattempt,https://i.redd.it/ujxthsw0umw41.jpg,832,NaN,1.588548e+09,suicide
5371,5372,to stop suicide,59371,iu8k8c,therewasanattempt,https://i.redd.it/2ez8ce09sln51.jpg,1212,NaN,1.600303e+09,suicide
5425,5426,To be edgy and poetic,56238,ceqsgz,therewasanattempt,https://i.redd.it/nnnp8d4oi1b31.jpg,497,NaN,1.563446e+09,suicide
5378,5379,"There was an attempt to ""shame"" someone who wa...",54025,freqp1,therewasanattempt,https://i.redd.it/4s7uadbduop41.jpg,627,NaN,1.585521e+09,suicide
5524,5525,To show China in a positive way,52816,fbu4hc,therewasanattempt,https://i.redd.it/67suqchmd2k41.jpg,547,NaN,1.583070e+09,suicide
5373,5374,..at reducing suicide rates..,46489,c371vx,therewasanattempt,https://i.imgur.com/eM27GT8.jpg,1352,NaN,1.561098e+09,suicide
6078,6079,West Taiwan really is a trainwreck.,45504,nnbcw3,dankmemes,https://i.redd.it/8lrwhca4dy171.jpg,885,NaN,1.622247e+09,suicide


In [5]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random 
import numpy as np
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/panjiayi/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [6]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [7]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [8]:
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer 

In [9]:
def process_tweet(tweet):
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)
    tweet2 = re.sub(r'https?://[^\s\n\r]+', '', tweet2)
    tweet2 = re.sub(r'#', '', tweet2)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet2)
    stopwords_english = stopwords.words('english')
    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
    stemmer = PorterStemmer() 
    # Create an empty list to store the stems
    tweets_stem = [] 
    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
    return tweets_stem

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs

freqs = build_freqs(train_x, train_y)  #Build Frequancy

def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE ###
    # calculate the sigmoid of z
    h = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return h

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):
        # get z, the dot product of x and theta
        z = x@theta
        # get the sigmoid of z
        h = sigmoid(z)   #python自变量是标量还是向量无影响
        # calculate the cost function
        J = (-1/m)*(np.transpose(y)@np.log(h)+np.transpose((1-y))@np.log(1-h))  #向量运算要用@
        # update the weights theta
        theta = theta-alpha/m*(np.transpose(x)@(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J,theta


def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements for [bias, positive, negative] counts
    x = np.zeros(3) 
    
    # bias term is set to 1
    x[0] = 1 
    
    ### START CODE HERE ###
    
    # loop through each word in the list of words
    for word in word_l:
        if (word, 1) in freqs:
            # increment the word count for the positive label 1
            x[1] += freqs[(word,1)]
        if (word, 0) in freqs:
            # increment the word count for the negative label 0
            x[2] += freqs[(word,0)]
        
    ### END CODE HERE ###
    
    x = x[np.newaxis, :]  # adding batch dimension for further processing
    assert(x.shape == (1, 3))
    return x



In [10]:
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

/var/folders/ss/sv50jmf54z5fg7h09y4ws6gc0000gn/T/ipykernel_32583/2023382314.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [11]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs, process_tweet=process_tweet)
    
    # make the prediction using x and theta
    y_pred = sigmoid(x@theta)
    
    ### END CODE HERE ###
    
    return y_pred

In [12]:
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE ###
    
    # the list for storing predictions
    y_hat = list()
    
    for tweet in test_x:
        # get the label prediction for the tweet
        z=extract_features(tweet, freqs, process_tweet=process_tweet)@theta
        y_pred = sigmoid(z)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy =np.sum(y_hat==test_y.flatten())/len(test_x)

    ### END CODE HERE ###
    
    return accuracy

In [13]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


In [14]:
#0  iloc[4]:"to stop suicide"
submission = reddit.submission(url="https://www.reddit.com/r/therewasanattempt/comments/iu8k8c/to_stop_suicide/")   #Find the first submission in 'suicide.csv' document                                                     
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of first submission    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
print(filtered_id[0:10])    #We are going to analyze 10 authors who were influenced by the first submission

[Redditor(name='coderedninja'), Redditor(name='RedMirricat'), Redditor(name='hadToPutDickinCatHmm'), Redditor(name='Defy19'), Redditor(name='yennifluent'), Redditor(name='yeahwellokay'), Redditor(name='thistyviolin'), Redditor(name='ViciousNakedMoleRat'), Redditor(name='gsychopato'), Redditor(name='Deja__Vu__')]


In [15]:
DIF0=[]
Before0=[]
After0=[]
for i in range(0, 1000):
    try:
        # Try to get some information from the author
        comment_count = reddit.redditor(filtered_id[i].name).comments.new(limit=1)
        # If successful, going on
        comment0 = []
        for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
            comment0.append([submission.title, 
                             submission.created,
                             i])
    except Exception as e:
        # If not, the author has suspended
        print(f"Skipping user (possible suspension): {filtered_id[i].name}")
        continue
    comment0 = pd.DataFrame(comment0,
                            columns=['title','created','Serial Number'])
    #comment0 = comment0.sort_values(by='created', ascending=True)
    split_time=posts['created'].iloc[4]
    comment0_before=comment0.loc[comment0['created']<split_time]
    comment0_after=comment0.loc[comment0['created']>split_time]
    text0_before = ''.join(comment0_before.title.values)
    y0_before=predict_tweet(text0_before, freqs, theta)
    text0_after = ''.join(comment0_after.title.values)
    y0_after=predict_tweet(text0_after, freqs, theta)
    difference0=(y0_before-y0_after)
    DIF0.append(difference0)
    Before0.append(comment0_before)
    After0.append(comment0_after)
    #print("Positiveness before the first submission is",y0_before,"and after the first submission is",y0_after,".\n","The difference is", difference0)

Skipping user (possible suspension): allthewrongwalls
Skipping user (possible suspension): Etherius
Skipping user (possible suspension): TimmyTesticles
Skipping user (possible suspension): sylphyyyy
Skipping user (possible suspension): Summamabitch
Skipping user (possible suspension): RevengeOTheCaptnsPen
Skipping user (possible suspension): Llanos31
Skipping user (possible suspension): Lilytrap
Skipping user (possible suspension): pujijik
Skipping user (possible suspension): jesus_zombie_attack
Skipping user (possible suspension): cocodecococo
Skipping user (possible suspension): ZippZappZippty
Skipping user (possible suspension): gmonkey143
Skipping user (possible suspension): Same-Cartographer488
Skipping user (possible suspension): LiquidMotion
Skipping user (possible suspension): T1nkyWinky
Skipping user (possible suspension): sheepeses
Skipping user (possible suspension): Ibeprasin
Skipping user (possible suspension): loveandcustard
Skipping user (possible suspension): audion00ba

In [16]:
len(DIF0)

858

In [19]:
#1  iloc[20]:"to prevent suicide"
submission = reddit.submission(url="https://www.reddit.com/r/therewasanattempt/comments/16psiru/to_prevent_suicide/")   #Find the second submission in 'suicide.csv' document     

In [20]:
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of second submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
#print(filtered_id[0:10])    #We are going to analyze 10 authors who were influenced by the second submission
DIF1=[]
Before1=[]
After1=[]
for i in range(0, 1000):
    try:
        # Try to get some information from the author
        comment_count = reddit.redditor(filtered_id[i].name).comments.new(limit=1)
        # If successful, going on
        comment1 = []
        for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
            comment1.append([submission.title, 
                             submission.created,
                             i])
    except Exception as e:
        # If not, the author has suspended
        #print(f"Skipping user (possible suspension): {filtered_id[i].name}")
        continue
        
    comment1 = pd.DataFrame(comment1,
                            columns=['title','created','Serial Number'])
    #comment1 = comment1.sort_values(by='created', ascending=True)
    split_time=posts['created'].iloc[20]
    comment1_before=comment1.loc[comment1['created']<split_time]
    comment1_after=comment1.loc[comment1['created']>split_time]
    text1_before = ''.join(comment1_before.title.values)
    y1_before=predict_tweet(text1_before, freqs, theta)
    text1_after = ''.join(comment1_after.title.values)
    y1_after=predict_tweet(text1_after, freqs, theta)
    difference1=(y1_before-y1_after)
    DIF1.append(difference1)
    Before1.append(comment1_before)
    After1.append(comment1_after)
    #print("Positiveness before the second submission is",y1_before,"and after the second submission is",y1_after,".\n","The difference is", difference1)


In [21]:
len(DIF1)

490

In [22]:
#2  loc[47]:"'If you were actually suicidal you'd have killed yourself already and not tell anybody' FUCK OFFFFFFF"
submission = reddit.submission(url="https://www.reddit.com/r/SuicideWatch/comments/jnrz57/if_you_were_actually_suicidal_youd_have_killed/")   #Find the third submission in 'suicide.csv' document 

In [23]:
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of third submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
#print(filtered_id[0:10])    
DIF2=[]
Before2=[]
After2=[]
for i in range(0, 1000):
    try:
        # Try to get some information from the author
        comment_count = reddit.redditor(filtered_id[i].name).comments.new(limit=1)
        # If successful, going on
        comment2 = []
        for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
            comment2.append([submission.title, 
                             submission.created,
                             i])
    except Exception as e:
        # If not, the author has suspended
        #print(f"Skipping user (possible suspension): {filtered_id[i].name}")
        continue
    comment2 = pd.DataFrame(comment2,
                            columns=['title','created','Serial Number'])
    #comment2 = comment2.sort_values(by='created', ascending=True)
    split_time=posts_sub['created'].loc[47]
    comment2_before=comment2.loc[comment2['created']<split_time]
    comment2_after=comment2.loc[comment2['created']>split_time]
    text2_before = ''.join(comment2_before.title.values)
    y2_before=predict_tweet(text2_before, freqs, theta)
    text2_after = ''.join(comment2_after.title.values)
    y2_after=predict_tweet(text2_after, freqs, theta)
    difference2=(y2_before-y2_after)
    DIF2.append(difference2)
    Before2.append(comment2_before)
    After2.append(comment2_after)
    #print("Positiveness before the third submission is",y2_before,"and after the third submission is",y2_after,".\n","The difference is", difference2)

In [24]:
len(DIF2)

103

In [25]:
#3  loc[3]:"I'm a fully-functioning suicidal."
submission = reddit.submission(url="https://www.reddit.com/r/SuicideWatch/comments/ppjowo/im_a_fullyfunctioning_suicidal/")   #Find the fourth submission in 'suicide.csv' document 

In [26]:
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of fourth submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
#print(filtered_id[0:10])  
Before3=[]
After3=[]
DIF3=[]
for i in range(0, 1000):
    try:
        # Try to get some information from the author
        comment_count = reddit.redditor(filtered_id[i].name).comments.new(limit=1)
        # If successful, going on
        comment3 = []
        for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
            comment3.append([submission.title,  
                             submission.created,
                             i])
    except Exception as e:
        # If not, the author has suspended
        #print(f"Skipping user (possible suspension): {filtered_id[i].name}")
        continue
    comment3 = pd.DataFrame(comment3,
                            columns=['title','created','Serial Number'])
    #comment3 = comment3.sort_values(by='created', ascending=True)
    split_time=posts_sub['created'].loc[3]
    comment3_before=comment3.loc[comment3['created']<split_time]
    comment3_after=comment3.loc[comment3['created']>split_time]
    text3_before = ''.join(comment3_before.title.values)
    y3_before=predict_tweet(text3_before, freqs, theta)
    text3_after = ''.join(comment3_after.title.values)
    y3_after=predict_tweet(text3_after, freqs, theta)
    difference3=(y3_before-y3_after)
    DIF3.append(difference3)
    Before3.append(comment3_before)
    After3.append(comment3_after)
    #print("Positiveness before the fourth submission is",y3_before,"and after the fourth submission is",y3_after,".\n","The difference is", difference3)

In [27]:
len(DIF3)

144

In [36]:
max_length = max(len(DIF0), len(DIF1), len(DIF2), len(DIF3))

# Pad the lists with None or NaN to make them of equal length
DIF0 += [None] * (max_length - len(DIF0))
DIF1 += [None] * (max_length - len(DIF1))
DIF2 += [None] * (max_length - len(DIF2))
DIF3 += [None] * (max_length - len(DIF3))
data={'DIF0':DIF0,
      'DIF1':DIF1,
      'DIF2':DIF2,
      'DIF3':DIF3}
df = pd.DataFrame(data)
df.to_csv('Difference.csv')

In [37]:
df

,DIF0,DIF1,DIF2,DIF3
0,[[0.008745052811056364]],[[-0.033839677468399776]],[[0.0023252623631968006]],[[-0.005110852959371859]]
1,[[-0.1026466422107003]],[[-0.11223124551259583]],[[0.179672007826861]],[[-0.08411560641561405]]
2,[[-0.06888551598445142]],[[-0.1312737284238924]],[[-0.0206910387020246]],[[-0.022560049469898036]]
3,[[0.04562360967435736]],[[-0.018278326891021934]],[[-0.0014545788500157641]],[[0.463627223014264]]
4,[[-0.012557223564133413]],[[-0.10143077476837875]],[[0.007673561260071293]],[[0.04525342956343281]]
...,...,...,...,...
853,[[0.04517484889126322]],None,None,None
854,[[-0.2557765473251503]],None,None,None
855,[[-0.24047055424481634]],None,None,None
856,[[0.25664047594108425]],None,None,None


In [38]:
from empath import Empath
lexicon = Empath()

In [39]:
#negative
lexicon.create_category("angry", ["angry"], model="reddit")
lexicon.create_category("sad", ["sad"], model="reddit")
lexicon.create_category("worried", ["worried"], model="reddit")
lexicon.create_category("frustrated", ["frustrated"], model="reddit")
lexicon.create_category("anxious", ["anxious"], model="reddit")
lexicon.create_category("nervous", ["nervous"], model="reddit")
lexicon.create_category("disappointed", ["disappointed"], model="reddit")
lexicon.create_category("gloomy", ["gloomy"], model="reddit")
lexicon.create_category("miserable", ["miserable"], model="reddit")
lexicon.create_category("lonely", ["lonely"], model="reddit")
#positive
lexicon.create_category("happy", ["happy"], model="reddit")
lexicon.create_category("loved", ["loved"], model="reddit")
lexicon.create_category("joyful", ["joyful"], model="reddit")
lexicon.create_category("content", ["content"], model="reddit")
lexicon.create_category("pleased", ["pleased"], model="reddit")
lexicon.create_category("grateful", ["grateful"], model="reddit")
lexicon.create_category("relieved", ["relieved"], model="reddit")
lexicon.create_category("optimistic", ["optimistic"], model="reddit")
lexicon.create_category("satisfied", ["satisfied"], model="reddit")
lexicon.create_category("excited", ["excited"], model="reddit")

["~~people~~", "angry", "shouting"]
["sad", "&gt;_&gt", "kidding", "&gt;.&gt", "&gt;_&gt", "&lt;.&lt"]
["worried", "worried", "concerned", "apprehensive", "worry", "afraid", "scared", "worrying", "worries", "Worried", "hesitant", "nervous", "biggest_worry", "cautious", "concerned", "worrying", "paranoid", "worrisome", "wary", "leery", "worry", "upset", "big_deal", "upsetting", "biggest_concern", "main_concern", "stressing", "main_worry", "unsure", "bothering", "bad_sign", "big_concern", "terrified", "last_thing", "upset", "wondering", "only_concern", "real_concern", "thrilled", "hurt", "b/c", "hurting", "second_thoughts", "huge_deal", "good_sign", "only_worry", "terrified", "antsy", "bothered", "surprised", "major_concern", "scared", "biggest_fear", "fussed", "Worried", "too_big_a_deal", "curious", "bigger_concern", "wondering", "would't", "only_fear", "guessing", "huge_concern", "insistent", "that_big_a_deal", "stressed", "trouble", "main_fear", "risk", "bother", "careful", "how_much_

In [40]:
columns=['before','after','difference']
index=["angry","sad","worried",
       "frustrated","anxious",
        "nervous","disappointed",
      "gloomy","miserable","longly",
       "happy", "loved", "joyful",
        "content", "pleased",
      "grateful", "relieved",
      "optimistic", "satisfied",
        "excited"]

In [44]:
dic0={}
for i in range(0,len(Before0)):
    df0=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
    for text in Before0[i].title:
        negative_before0 = lexicon.analyze(text, categories=["angry","sad","worried",
                                                    "frustrated","anxious",
                                                    "nervous","disappointed",
                                                    "gloomy","miserable","longly"])
        positive_before0 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                     "content", "pleased",
                                                     "grateful", "relieved",
                                                     "optimistic", "satisfied",
                                                     "excited", "loved"])
        for key, value in positive_before0.items():
            df0.loc[key,'before']+=value
        for key, value in negative_before0.items():
            df0.loc[key,'before']+=value
    for text in After0[i].title:
        negative_after0 = lexicon.analyze(text, categories=["angry", "sad", "worried",
                                                             "frustrated", "anxious",
                                                             "nervous", "disappointed",
                                                             "gloomy", "miserable", "longly"])
        positive_after0 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                             "content", "pleased",
                                                             "grateful", "relieved",
                                                             "optimistic", "satisfied",
                                                             "excited"])

        for key, value in positive_after0.items():
            df0.loc[key,'after']+=value
        for key, value in negative_after0.items():
            df0.loc[key,'after']+=value
    df0.difference=df0.before-df0.after
    dic0[i]=df0

In [1]:
dic0[1]

NameError: name 'dic0' is not defined

In [46]:
dic1={}
for i in range(0,len(Before1)):
    df1=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
    for text in Before1[i].title:
        negative_before1 = lexicon.analyze(text, categories=["angry","sad","worried",
                                                    "frustrated","anxious",
                                                    "nervous","disappointed",
                                                    "gloomy","miserable","longly"])
        positive_before1 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                     "content", "pleased",
                                                     "grateful", "relieved",
                                                     "optimistic", "satisfied",
                                                     "excited", "loved"])
        for key, value in positive_before1.items():
            df1.loc[key,'before']+=value
        for key, value in negative_before1.items():
            df1.loc[key,'before']+=value
    for text in After1[i].title:
        negative_after1 = lexicon.analyze(text, categories=["angry", "sad", "worried",
                                                             "frustrated", "anxious",
                                                             "nervous", "disappointed",
                                                             "gloomy", "miserable", "longly"])
        positive_after1 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                             "content", "pleased",
                                                             "grateful", "relieved",
                                                             "optimistic", "satisfied",
                                                             "excited"])

        for key, value in positive_after1.items():
            df1.loc[key,'after']+=value
        for key, value in negative_after1.items():
            df1.loc[key,'after']+=value
    df1.difference=df1.before-df1.after
    dic1[i]=df1

In [47]:
dic1[0]

,before,after,difference
angry,0.0,0.0,0.0
sad,0.0,0.0,0.0
worried,0.0,0.0,0.0
frustrated,0.0,0.0,0.0
anxious,0.0,0.0,0.0
nervous,0.0,0.0,0.0
disappointed,0.0,0.0,0.0
gloomy,0.0,0.0,0.0
miserable,0.0,0.0,0.0
longly,0.0,0.0,0.0


In [48]:
dic2={}
for i in range(0,len(Before2)):
    df2=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
    for text in Before2[i].title:
        negative_before2 = lexicon.analyze(text, categories=["angry","sad","worried",
                                                    "frustrated","anxious",
                                                    "nervous","disappointed",
                                                    "gloomy","miserable","longly"])
        positive_before2 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                     "content", "pleased",
                                                     "grateful", "relieved",
                                                     "optimistic", "satisfied",
                                                     "excited", "loved"])
        for key, value in positive_before2.items():
            df2.loc[key,'before']+=value
        for key, value in negative_before2.items():
            df2.loc[key,'before']+=value
    for text in After2[i].title:
        negative_after2 = lexicon.analyze(text, categories=["angry", "sad", "worried",
                                                             "frustrated", "anxious",
                                                             "nervous", "disappointed",
                                                             "gloomy", "miserable", "longly"])
        positive_after2 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                             "content", "pleased",
                                                             "grateful", "relieved",
                                                             "optimistic", "satisfied",
                                                             "excited"])

        for key, value in positive_after2.items():
            df2.loc[key,'after']+=value
        for key, value in negative_after2.items():
            df2.loc[key,'after']+=value
    df2.difference=df2.before-df2.after
    dic2[i]=df2

In [49]:
dic2[1]

,before,after,difference
angry,0.0,0.0,0.0
sad,0.0,0.0,0.0
worried,1.0,2.0,-1.0
frustrated,0.0,1.0,-1.0
anxious,0.0,0.0,0.0
nervous,0.0,2.0,-2.0
disappointed,3.0,0.0,3.0
gloomy,1.0,0.0,1.0
miserable,1.0,0.0,1.0
longly,0.0,0.0,0.0


In [50]:
dic3={}
for i in range(0,len(Before3)):
    df3=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
    for text in Before3[i].title:
        negative_before3 = lexicon.analyze(text, categories=["angry","sad","worried",
                                                    "frustrated","anxious",
                                                    "nervous","disappointed",
                                                    "gloomy","miserable","longly"])
        positive_before3 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                     "content", "pleased",
                                                     "grateful", "relieved",
                                                     "optimistic", "satisfied",
                                                     "excited", "loved"])
        for key, value in positive_before3.items():
            df3.loc[key,'before']+=value
        for key, value in negative_before3.items():
            df3.loc[key,'before']+=value
    for text in After3[i].title:
        negative_after3 = lexicon.analyze(text, categories=["angry", "sad", "worried",
                                                             "frustrated", "anxious",
                                                             "nervous", "disappointed",
                                                             "gloomy", "miserable", "longly"])
        positive_after3 = lexicon.analyze(text, categories=["happy", "loved", "joyful",
                                                             "content", "pleased",
                                                             "grateful", "relieved",
                                                             "optimistic", "satisfied",
                                                             "excited"])

        for key, value in positive_after3.items():
            df3.loc[key,'after']+=value
        for key, value in negative_after3.items():
            df3.loc[key,'after']+=value
    df3.difference=df3.before-df3.after
    dic3[i]=df3

In [51]:
dic3[1]

,before,after,difference
angry,0.0,0.0,0.0
sad,0.0,0.0,0.0
worried,1.0,0.0,1.0
frustrated,1.0,0.0,1.0
anxious,4.0,2.0,2.0
nervous,0.0,1.0,-1.0
disappointed,1.0,0.0,1.0
gloomy,0.0,0.0,0.0
miserable,4.0,0.0,4.0
longly,0.0,0.0,0.0


In [2]:
import json
converted_dic0 = {key: df.to_dict(orient='split') for key, df in dic0.items()}
with open('dic0.json', 'w') as json_file:
    json.dump(converted_dic0, json_file)
converted_dic1 = {key: df.to_dict(orient='split') for key, df in dic1.items()}
with open('dic1.json', 'w') as json_file:
    json.dump(converted_dic0, json_file)
converted_dic2 = {key: df.to_dict(orient='split') for key, df in dic2.items()}
with open('dic2.json', 'w') as json_file:
    json.dump(converted_dic0, json_file)
converted_dic3 = {key: df.to_dict(orient='split') for key, df in dic3.items()}
with open('dic3.json', 'w') as json_file:
    json.dump(converted_dic0, json_file)

NameError: name 'dic0' is not defined

In [3]:
with open('dic0.json', 'r') as json_file:
    dic = json.load(json_file)

In [7]:
dic['2']

{'index': ['angry',
  'sad',
  'worried',
  'frustrated',
  'anxious',
  'nervous',
  'disappointed',
  'gloomy',
  'miserable',
  'longly',
  'happy',
  'loved',
  'joyful',
  'content',
  'pleased',
  'grateful',
  'relieved',
  'optimistic',
  'satisfied',
  'excited'],
 'columns': ['before', 'after', 'difference'],
 'data': [[0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 1.0, -1.0],
  [1.0, 1.0, 0.0],
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0],
  [0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0],
  [0.0, 1.0, -1.0],
  [0.0, 0.0, 0.0],
  [0.0, 1.0, -1.0],
  [0.0, 1.0, -1.0],
  [0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0],
  [0.0, 1.0, -1.0],
  [0.0, 0.0, 0.0],
  [0.0, 1.0, -1.0]]}

In [75]:
data0=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
data1=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
data2=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
data3=pd.DataFrame(np.zeros((20,3)),columns=columns,index=index)
for key in dic0:
    data0+=dic0[key]
for key in dic1:
    data1+=dic1[key]
for key in dic2:
    data2+=dic2[key]
for key in dic3:
    data3+=dic3[key]

In [76]:
data0

,before,after,difference
angry,13.0,30.0,-17.0
sad,41.0,34.0,7.0
worried,390.0,430.0,-40.0
frustrated,230.0,266.0,-36.0
anxious,144.0,140.0,4.0
nervous,294.0,334.0,-40.0
disappointed,288.0,295.0,-7.0
gloomy,98.0,130.0,-32.0
miserable,185.0,203.0,-18.0
longly,0.0,0.0,0.0


In [78]:
with pd.ExcelWriter('influence.xlsx') as writer:
    data0.to_excel(writer, sheet_name='Sheet0')
    data1.to_excel(writer, sheet_name='Sheet1')
    data2.to_excel(writer, sheet_name='Sheet2')
    data3.to_excel(writer, sheet_name='Sheet3')

In [ ]:
data1